In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
print('Pytorch version :', torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'device : [{device}]')

Pytorch version : 1.7.1
device : [cuda:0]


In [57]:
from torchvision import datasets, transforms
train_data = datasets.MNIST(root='./data',train=True,transform=transforms.ToTensor(),download=False)
test_data = datasets.MNIST(root='./data',train=False,transform=transforms.ToTensor(),download=False)

In [58]:
BATCH_SIZE = 256
train_iter = DataLoader(train_data,shuffle=True,batch_size=BATCH_SIZE,num_workers=1,drop_last=True)
test_iter = DataLoader(test_data,shuffle=True,batch_size=BATCH_SIZE,num_workers=1,drop_last=True)

In [15]:
train_iter

In [78]:
x_dim = 28
h_dim = 256
y_dim = 10
n_layer = 3
        
lin_xh_f = nn.Linear(x_dim,h_dim)
lin_hh_f = nn.Linear(h_dim,h_dim)
        
lin_xh_in1 = nn.Linear(x_dim,h_dim)
lin_hh_in1 = nn.Linear(h_dim,h_dim)
lin_xh_in2 = nn.Linear(x_dim,h_dim)
lin_hh_in2 = nn.Linear(h_dim,h_dim)
        
lin_xh_out = nn.Linear(x_dim,h_dim)
lin_hh_out = nn.Linear(h_dim,h_dim)     
# result
lin_out = nn.Linear(h_dim,y_dim)

h_0 = torch.zeros(h_dim)
c_0 = torch.zeros(h_dim)

loss = nn.CrossEntropyLoss()
optimizer1 = optim.Adam(lin_xh_f.parameters(),lr=1e-3)
optimizer2 = optim.Adam(lin_hh_f.parameters(),lr=1e-3)
optimizer3 = optim.Adam(lin_xh_in1.parameters(),lr=1e-3)
optimizer4 = optim.Adam(lin_hh_in1.parameters(),lr=1e-3)
optimizer5 = optim.Adam(lin_xh_in2.parameters(),lr=1e-3)
optimizer6 = optim.Adam(lin_hh_in2.parameters(),lr=1e-3)
optimizer7 = optim.Adam(lin_xh_out.parameters(),lr=1e-3)
optimizer8 = optim.Adam(lin_hh_out.parameters(),lr=1e-3)
optimizer9 = optim.Adam(lin_out.parameters(),lr=1e-3)

In [ ]:
class LSTM(nn.Module):
    def __init__(x_dim=28,h_dim=256,y_dim=10):
        super(LSTM,self).__init__()
        self.x_dim = x_dim
        self.h_dim = h_dim
        self.y_dim = y_dim
        sefl.lin_xh_f
        self.lin_xh_f = nn.Linear(x_dim,h_dim)
        lsef.lin_hh_f = nn.Linear(h_dim,h_dim)
        
        self.lin_xh_in1 = nn.Linear(x_dim,h_dim)
        self.lin_hh_in1 = nn.Linear(h_dim,h_dim)
        self.lin_xh_in2 = nn.Linear(x_dim,h_dim)
        self.lin_hh_in2 = nn.Linear(h_dim,h_dim)
        
        self.lin_xh_out = nn.Linear(x_dim,h_dim)
        self.lin_hh_out = nn.Linear(h_dim,h_dim)     
        # result
        self.lin_out = nn.Linear(h_dim,y_dim)
        
    def forward(self,x):
        h_0 = torch.zeros(h_dim)
        c_0 = torch.zeros(h_dim)
        h_n = h_0
        c_n = c_0
        for i in range(x_dim):
            # forget
            batch_in = batch_in_origin[:,i,:]
            a = lin_xh_f(batch_in)
            b = lin_hh_f(h_n)
            forget_out = F.sigmoid(a+b)

            # input
            d = lin_xh_in1(batch_in)
            e = lin_hh_in1(h_n)
            f = lin_xh_in2(batch_in)
            g = lin_hh_in2(h_n)
            input_out = F.sigmoid(d+e)*F.tanh(f+g)

            # output
            h = lin_xh_out(batch_in)
            i = lin_hh_out(h_n)
            output_out = F.sigmoid(h+i)

            # combine
            c_n = c_n*forget_out + input_out
            h_n = F.tanh(c_n)*output_out
        # result
        print(h_n.size())
        output = lin_out(h_n)

        

In [88]:
h_n = h_0
c_n = c_0
batch_1 = next(iter(train_iter))
batch_in_origin = batch_1[0].view(-1,28,28)
batch_out = batch_1[1]
batch_in.size()

torch.Size([256, 28])

In [94]:
    
for i in range(x_dim):
    # forget
    batch_in = batch_in_origin[:,i,:]
    a = lin_xh_f(batch_in)
    b = lin_hh_f(h_n)
    forget_out = F.sigmoid(a+b)

    # input
    d = lin_xh_in1(batch_in)
    e = lin_hh_in1(h_n)
    f = lin_xh_in2(batch_in)
    g = lin_hh_in2(h_n)
    input_out = F.sigmoid(d+e)*F.tanh(f+g)

    # output
    h = lin_xh_out(batch_in)
    i = lin_hh_out(h_n)
    output_out = F.sigmoid(h+i)

    # combine
    c_n = c_n*forget_out + input_out
    h_n = F.tanh(c_n)*output_out
# result
print(h_n.size())
output = lin_out(h_n)

# 256,1,28,28  
print(output)
loss_out = loss(output,batch_out)
print(loss_out)

torch.Size([256, 256])
tensor([[-0.0447, -0.0301,  0.0796,  ..., -0.0058, -0.0331,  0.0246],
        [-0.0428, -0.0083,  0.0739,  ..., -0.0039, -0.0388, -0.0030],
        [-0.0420, -0.0259,  0.0726,  ..., -0.0075, -0.0282,  0.0146],
        ...,
        [-0.0292, -0.0314,  0.0743,  ..., -0.0112, -0.0287, -0.0017],
        [-0.0410, -0.0249,  0.0776,  ..., -0.0076, -0.0284,  0.0078],
        [-0.0419, -0.0307,  0.0785,  ..., -0.0054, -0.0285,  0.0145]],
       grad_fn=<AddmmBackward>)
tensor(2.3025, grad_fn=<NllLossBackward>)


In [21]:
a = nn.Linear(2,3)
aa = a(torch.Tensor([1,2]))
bb = a(torch.Tensor([2,3]))
print(aa,bb)
aa*(bb)

tensor([-0.7501, -0.3321,  1.1901], grad_fn=<AddBackward0>) tensor([-1.3128, -0.7010,  2.3911], grad_fn=<AddBackward0>)


tensor([0.9848, 0.2328, 2.8457], grad_fn=<MulBackward0>)